In [1]:
import sys
import os
#dir_path = os.path.dirname(os.path.realpath(__file__))
#sys.path.append(dir_path)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import argparse
import scipy.io
import numpy as np
import deepxde as dde # version 0.11 or higher
from deepxde.backend import tf

Using backend: tensorflow.compat.v1

2023-02-17 16:02:57.830941: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 16:03:01.098168: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 16:03:01.098216: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 16:03:20.100209: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object 

Instructions for updating:
non-resource variables are not supported in the long term


2023-02-17 16:03:44.022084: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-17 16:03:44.022184: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cx3-6-29.cx3.hpc.ic.ac.uk): /proc/driver/nvidia/version does not exist


In [2]:
input = 4 # network input size 
num_hidden_layers = 5 # number of hidden layers for NN 
hidden_layer_size = 60 # size of each hidden layers 
output = 3 # network input size 

num_domain = 40000 # number of training points within the domain
num_boundary = 4000 # number of training boundary condition points on the geometry boundary
num_test = 1000 # number of testing points within the domain
MAX_MODEL_INIT = 16 # maximum number of times allowed to initialize the model
MAX_LOSS = 4 # upper limit to the initialized loss
epochs_init = 15000 # number of epochs for training initial phase
epochs_main = 150000 # number of epochs for main training phase
lr = 0.0005 # learning rate



In [3]:
a = 0.01
b = 0.15
D = 0.1
k = 8
mu_1 = 0.2
mu_2 = 0.3
epsilon = 0.002

## Geometry Parameters
min_x = 0.1
max_x = 10
min_y = 0.1
max_y = 10
min_x = 0.1
min_z = 0.1
max_z = 1
min_t = 1
max_t = 50
spacing = 0.1

In [4]:
file_name = "3D_1BCL_50ms.mat"
data = scipy.io.loadmat(file_name)
t, x, y, z, Vsav, Wsav = data["t"], data["x"], data["y"], data["z"], data["Vsav"], data["Wsav"]
X, T, Y, Z = np.meshgrid(x,t,y,z)
Y = Y.reshape(-1, 1)
Z = Z.reshape(-1, 1)

max_t = np.max(t)
max_x = np.max(x)        
X = X.reshape(-1, 1)
T = T.reshape(-1, 1)
V = Vsav.reshape(-1, 1)
W = Wsav.reshape(-1, 1)

In [5]:
observe_x = np.hstack((X, Y, Z, T))

In [6]:
def IC_func(observe_train, v_train):
        
        T_ic = observe_train[:,-1].reshape(-1,1)
        idx_init = np.where(np.isclose(T_ic,1))[0]
        v_init = v_train[idx_init]
        observe_init = observe_train[idx_init]
        return dde.PointSetBC(observe_init,v_init,component=0)

In [7]:
def boundary_func_3d_cube(x, on_boundary):
    return on_boundary
    #return on_boundary and ~(x[0:3]==[min_x,min_y,min_z]).all() and  ~(x[0:3]==[min_x,min_y, max_z]).all() and \
    #    ~(x[0:3]==[max_x,min_y,min_z]).all() and  ~(x[0:3]==[max_x,min_y,max_z]).all() and  ~(x[0:3]==[min_x,max_y,min_z]).all() \
    #    and  ~(x[0:3]==[min_x,max_y,max_z]).all() and  ~(x[0:3]==[max_x,max_y,min_z]).all() and  ~(x[0:3]==[max_x,max_y,max_z]).all()

In [8]:
def BC_func(geomtime):
    bc = dde.NeumannBC(geomtime, lambda x:  np.zeros((len(x), 1)), boundary_func_3d_cube, component=0)
    return bc

In [9]:
observe_train, observe_test, v_train, v_test, w_train, w_test = train_test_split(observe_x,V,W,test_size=0.8)   

In [10]:
geom = dde.geometry.Cuboid([min_x,min_y, min_z], [max_x,max_y, max_z])
timedomain = dde.geometry.TimeDomain(min_t, max_t)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

In [11]:
## Define Boundary Conditions
bc = BC_func(geomtime)
print(bc)

## Define Initial Conditions
ic = IC_func(observe_train, v_train)
print(ic)
    
## Model observed data
observe_v = dde.PointSetBC(observe_train, v_train, component=0)
input_data = [bc, ic, observe_v]

In [12]:
net = dde.maps.FNN([input] + [hidden_layer_size] * num_hidden_layers + [output], "tanh", "Glorot uniform")

In [13]:
def pde_3D(x, y):
    
    V, W = y[:, 0:1], y[:, 1:2]
    dv_dt = dde.grad.jacobian(y, x, i=0, j=3)
    dv_dxx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    dv_dyy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    dv_dzz = dde.grad.hessian(y, x, component=0, i=2, j=2)
    dw_dt = dde.grad.jacobian(y, x, i=1, j=3)

        ## Coupled PDE+ODE Equations
    eq_a = dv_dt -  D*(dv_dxx + dv_dyy + dv_dzz) + k*V*(V-a)*(V-1) +W*V 
    eq_b = dw_dt -  (epsilon + (mu_1*W)/(mu_2+V))*(-W -k*V*(V-b-1))
    return [eq_a, eq_b]

In [14]:
pde = pde_3D


In [15]:
print(num_domain)
print(num_boundary)
print(observe_train.shape)
print(num_test)
print(type(pde))
print(observe_train[0].size)
print(observe_train[1].size)
print(len(input_data))
print(input_data[1])
print(bc)

40000
4000
(600000, 4)
1000
<class 'function'>
4
4
3


In [16]:
pde_data = dde.data.TimePDE(geomtime, pde, input_data,
                            num_domain = num_domain, 
                            num_boundary=num_boundary, 
                            anchors=observe_train,
                            num_test=num_test) 
model = dde.Model(pde_data, net)

In [17]:
def stable_init(model):
    ## Stabalize initialization process by capping the losses
    losshistory, _ = model.train(epochs=1)
    initial_loss = max(losshistory.loss_train[0])
    num_init = 1
    while initial_loss>MAX_LOSS or np.isnan(initial_loss):
        num_init += 1
        model = dde.Model(pde_data, net)
        model.compile("adam", lr=lr)
        losshistory, _ = model.train(epochs=1)
        initial_loss = max(losshistory.loss_train[0])
        if num_init > MAX_MODEL_INIT:
            raise ValueError('Model initialization phase exceeded the allowed limit')
    return 0

In [18]:
out_path = 'test.txt'

In [19]:
def train_3_phase(self, out_path):
    init_weights = [0,0,0,0,1]
        
    ## Initial phase
    model.compile("adam", lr=0.0005, loss_weights=init_weights)
    losshistory, train_state = model.train(epochs=epochs_init, model_save_path = out_path)
    ## Main phase
    model.compile("adam", lr=lr)
    losshistory, train_state = model.train(epochs=epochs_main, model_save_path = out_path)
    ## Final phase
    model.compile("L-BFGS-B")
    losshistory, train_state = model.train(model_save_path = out_path)
    return losshistory, train_state

In [27]:
def train_1_phase(model, out_path):
    
    losshistory, train_state = model.train(epochs=epochs_main, model_save_path = out_path)
    return losshistory, train_state

In [30]:
def train(model, out_path):
        
        ## Stabalize initialization process by capping the losses
        stable_init(model)
      
     
        losshistory, train_state = train_1_phase(model, out_path)
                   
        return model, losshistory, train_state

In [26]:
print(type(model))

<class 'deepxde.model.Model'>


In [31]:
model, losshistory, train_state = train(model, out_path)

Initializing variables...


AttributeError: 'NoneType' object has no attribute 'run'